In [3]:
!pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/20/34/a08b0ee99715eaba118cbe19a71f7b5e2425c2718ef96007c325944a1152/datasets-3.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=15.0.0 from https://files.pythonhosted.org/packages/bb/1b/2168d6050e52ff1e6cefc61d600723870bf569cbf41d13db939c8cf97a16/pyarrow-20.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Using cached pyarrow-20.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee34/dill-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/11/a7/81dba5010f7e733de88af9555725146fc133be97ce36533867f4c7e75066/xxhash-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for

In [6]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import torch
import numpy as np

In [24]:
ds = load_dataset("SetFit/tweet_sentiment_extraction")
ds

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['textID', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['textID', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [25]:
print(ds["train"].features)

{'textID': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'label_text': Value(dtype='string', id=None)}


In [26]:
set(ds['train']['label'])

{0, 1, 2}

In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
text = "puppy is cute"
input = tokenizer(text,return_tensors='pt',padding=True, truncation=True,max_length =512)
output = model(**input)
predicted_class = torch.argmax(output.logits).item()
print(f"Predicted Sentiment: {model.config.id2label[predicted_class]}")

Predicted Sentiment: LABEL_1


In [40]:
for name, module in model.named_modules():
    print(name)


bert
bert.embeddings
bert.embeddings.word_embeddings
bert.embeddings.position_embeddings
bert.embeddings.token_type_embeddings
bert.embeddings.LayerNorm
bert.embeddings.dropout
bert.encoder
bert.encoder.layer
bert.encoder.layer.0
bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.0.intermediate
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.intermediate.intermediate_act_fn
bert.encoder.layer.0.output
bert.encoder.layer.0.output.dense
bert.encoder.layer.0.output.LayerNorm
bert.encoder.layer.0.output.dropout
bert.encoder.layer.1
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.e

In [41]:
peft_config = LoraConfig(
                    task_type = 'SEQ_CLS',
                    r = 5,
                    lora_alpha = 42,
                    lora_dropout = 0.01,
                    target_modules = ["query", "value"])
model = get_peft_model(model,peft_config)

In [39]:
help(LoraConfig)

Help on class LoraConfig in module peft.tuners.lora.config:

class LoraConfig(peft.config.PeftConfig)
 |  LoraConfig(task_type: Union[str, peft.utils.peft_types.TaskType, NoneType] = None, peft_type: Union[str, peft.utils.peft_types.PeftType, NoneType] = None, auto_mapping: Optional[dict] = None, base_model_name_or_path: Optional[str] = None, revision: Optional[str] = None, inference_mode: bool = False, r: 'int' = 8, target_modules: 'Optional[Union[list[str], str]]' = None, exclude_modules: 'Optional[Union[list[str], str]]' = None, lora_alpha: 'int' = 8, lora_dropout: 'float' = 0.0, fan_in_fan_out: 'bool' = False, bias: "Literal['none', 'all', 'lora_only']" = 'none', use_rslora: 'bool' = False, modules_to_save: 'Optional[list[str]]' = None, init_lora_weights: "bool | Literal['gaussian', 'eva', 'olora', 'pissa', 'pissa_niter_[number of iters]', 'corda', 'loftq']" = True, layers_to_transform: 'Optional[Union[list[int], int]]' = None, layers_pattern: 'Optional[Union[list[str], str]]' = No

In [ ]:
model.print_trainable_parameters()

trainable params: 186,627 || all params: 109,671,174 || trainable%: 0.1702


In [52]:
from sklearn.model_selection import train_test_split
train = ds['train'].train_test_split(test_size=0.1, seed=42)['test']  # 10% sample
test = ds['test'].train_test_split(test_size=0.1, seed=42)['test']

In [53]:
print(train,test)

Dataset({
    features: ['textID', 'text', 'label', 'label_text'],
    num_rows: 2749
}) Dataset({
    features: ['textID', 'text', 'label', 'label_text'],
    num_rows: 354
})


In [64]:
tokenized_train = tokenizer(train['text'],return_tensors='pt',padding=True, truncation=True,max_length =512)
tokenized_test = tokenizer(test['text'],return_tensors='pt',padding=True, truncation=True,max_length =512)

In [65]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

In [66]:
train_dataset = SentimentDataset(tokenized_train, train['label'])
val_dataset = SentimentDataset(tokenized_test, test['label'])

In [68]:
train_dataset[0]

{'input_ids': tensor([  101,  2054,  4148,  2000,  1996, 29229,  1997,  6737,  1029,  8046,
          2320,  2153,  1029,  1012,  1012,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [54]:
from sklearn.metrics import accuracy_score
def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}


In [58]:
len(train['text'])

2749

In [60]:
len(train['label'])

2749

In [ ]:
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=100,
    eval_steps=100
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_datset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_351240/377984086.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [62]:
try:
    trainer.train()
except Exception as e:
    print(f"Error during raining: {e}")
    raise

Error during raining: 'list' object has no attribute 'keys'


AttributeError: 'list' object has no attribute 'keys'